In [2]:

import time
import numpy as np
import math 
import cv2
import matplotlib.pyplot as plt
import opencv_jupyter_ui as jcv2

## Libraries 
from src.Motion_Control import thymio as th
from src.Global_Nav import helpers_global as gb
from src.Vision import vision as vs
#import filtering 
from src.Local_Nav import psymap as pm  
from src.Local_Nav import local_navigation as ln
from src.Filtering import filtering
import PID 



For the path planning we choose the A\* Algorithm with the occupancy grid given by the vision module. We choosed to use A\* as it is the simplest algorithm to have the optimal path
To have a margin to the obtacle we modified the cost of the cell by multipliying it by 2 if the cell is in a radius of 3 cells from an obstacle and by 1.5 if the cell is in a radius of 5 cell from the obstacle.

We also used the Douglas Peucker algorithm to simplify the path given by A\*, and have a smoother trajectory. 
Finally, we decided to use a path containing only the points where the direction is changing.


## **Global Naviguation**

All the global naviguation function are grouped in the folder `helpers_global`.




### A\* computation
Since we are working with a grid filled with 1 for global obstacles and 0 for free cells, we are using the same A\* algorithm as the one in the exercise on Path Planning. We made a couple changes to adapt it to the project. Note that the grid computed by the vision is taking into account the size of the robot so the obstacles are bigger than in reality. After getting, the start and goal position from the camera, we can use the  `global_final` function to get the path that is already simplified by the Douglas Peucker algorithm. It also use a VISUALIZE parameter that plot the path computed on the given grid. To naviguate further from the global obstacles, we added a cost on the gScore if the cell is near an obstacle. We computed an safety margin that multiplies the current cost (the default one is 1) that is add to gScore : 

<small>

```python
def calculate_safety_margin(neighbor, occupancy_grid):
    distance_threshold_big = 5  
    distance_threshold_small = 3
    distance_to_obstacle = distance_to_nearest_obstacle(neighbor, occupancy_grid)

    if distance_to_obstacle < distance_threshold_big:
        if distance_to_obstacle < distance_threshold_small:
            safety_margin = 2     
        else: 
            safety_margin = 1.5  
    else:
        safety_margin = 1.0  

    return safety_margin

```

</small>

where `distance_to_nearest_obstacle` gives the minimal distance to any obstacles in the grid


### Douglas-Peucker Algorithm

Since the path given by A\* is giving points on the grid and therefore is often changing in direction we simplified it by the Douglas Peucker Algorithm:

<small>

```python
def douglas_peucker(coords, epsilon):
    if len(coords) <= 2:
        return [coords[0], coords[-1]]

    dmax = 0
    index = 0
    end = len(coords) - 1
    for i in range(1, end):
        d = point_to_line_distance(coords[i], coords[0], coords[end])
        if d > dmax:
            index = i
            dmax = d

    if dmax > epsilon:
        results1 = douglas_peucker(coords[:index + 1], epsilon)
        results2 = douglas_peucker(coords[index:], epsilon)

        results = results1[:-1] + results2
    else:
        results = [coords[0], coords[end]]

    return results
```

</small>

It is a recursive algorithm that take the path in argument and give a simplified one that only keep the point with big variation in between



In [6]:
from tdmclient import ClientAsync
client = ClientAsync()
node = await client.wait_for_node() #_ = protected #__ = private = shouldn't access node outside of the class
await node.lock()

Node 7648c768-a4cd-4427-9998-204136028638

In [7]:
## Constant 
MAP_SHAPE = (1000,700)
REFRAME = True 
TS =0.01
EPSILON_ANGLE= np.pi/4
VISUALIZE = True
MAP_SHAPE_MM = (1000,700)
MAP_SHAPE_CELL = (50,35)
ROBROAD = 110
SIMPLIFY = 0.8

In [8]:
await th.get_proximity_ground_values(client)

[997, 998]

In [9]:
#Load the camera : 25sec

cap = cv2.VideoCapture(0)


In [10]:
path, state_estimation_prev2, P_estimation_prev = th.init(cap, REFRAME, MAP_SHAPE, VISUALIZE,ROBROAD, SIMPLIFY )

KeyboardInterrupt: 

In [9]:
#setup
if REFRAME:
    Tmap = vs.get_warp(cap,MAP_SHAPE_MM,20,10)

ret,frame = cap.read()
if ret:
    if REFRAME:
        frame = cv2.warpPerspective(frame,Tmap,MAP_SHAPE_MM)
    fmap = vs.get_grid_fixed_map(frame,MAP_SHAPE_CELL, robrad=50)
    obscont = vs.get_obstacles(frame)
    print("Searching for destination...")
    while True:
        ret,frame = cap.read()
        if ret:
            if REFRAME:
                frame = cv2.warpPerspective(frame,Tmap,MAP_SHAPE_MM)
            ret,destmm = vs.get_destination(frame)
            if ret:
                dest = gb.convert_to_idx([coord / 10.0 for coord in destmm],2)
                dest[1]= 35-dest[1]
                dest = tuple(dest)
                break
            else:
                cv2.imshow()
        else:
            print("No camera !")
            break
    print("Found destination Point at {} [mm] {} [cells]".format(destmm,dest))
    print("Searching for Robot...")
    while True:
        ret,frame = cap.read()
        if ret:
            if REFRAME:
                frame = cv2.warpPerspective(frame,Tmap,MAP_SHAPE_MM)
            hls_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HLS_FULL) 
            ret,robpos,orient,pxpcm = vs.get_Robot_position_orientation(hls_frame)
            if ret:
                print("Robot found at {} [mm], {} [rad]".format(robpos,orient))
                break
        else:
            print("No camera !")
            break

start = gb.convert_to_idx(robpos,20)
start[1]= MAP_SHAPE_CELL[1]-start[1]
start = tuple(start)
path = gb.global_final(fmap,start,dest, "8N", VISUALIZE)

state_estimation_prev2 = np.array([[robpos[0]],[700-robpos[1]], [orient], [0], [0]])
P_estimation_prev =  np.diag([100, 100, 0.75, 10, 0.75])


Searching for destination...


error: OpenCV(4.8.1) :-1: error: (-5:Bad argument) in function 'imshow'
> Overload resolution failed:
>  - imshow() missing required argument 'winname' (pos 1)
>  - imshow() missing required argument 'winname' (pos 1)
>  - imshow() missing required argument 'winname' (pos 1)


In [ ]:
start = gb.convert_to_idx(robpos,20)
start[1]= MAP_SHAPE_CELL[1]-start[1]
start = tuple(start)
path = gb.global_final(fmap,start,dest, "8N", VISUALIZE)

state_estimation_prev2 = np.array([[robpos[0]],[700-robpos[1]], [orient], [0], [0]])
P_estimation_prev =  np.diag([100, 100, 0.75, 10, 0.75])

In [18]:
##Main boucle with Kalman Aubin

local_obstacle = False
counter=0
record= []

check=[]
check_point_prev=np.array([0,0])

start_time = time.time()
kidnapping_state = False

state_estimation_prev= state_estimation_prev2
Tmap = vs.get_warp(cap,MAP_SHAPE,10,1)
while True:
    
    

    ret,frame = cap.read()
    if ret : 
        
        # maps capture to map
        if REFRAME:
            frame = cv2.warpPerspective(frame,Tmap,MAP_SHAPE)
        # maps BGR to HLS color space for simplicity
        HLS = cv2.cvtColor(frame, cv2.COLOR_BGR2HLS_FULL)

        bool_pos ,center,orient, scale = vs.get_Robot_position_orientation(HLS, 5)
        if bool_pos : 
            #center = center/20.0
            center[1] = 700- center[1]
            orient2 = orient
            if orient <0:
                orient = orient +2*np.pi
            x_est_cam = np.array([center[0], center[1], orient])
            marre = np.array([x_est_cam[0]/20, x_est_cam[1]/20, orient2])
            record.append(marre)
            
        else :
            x_est_cam = None
        ground_values = await th.get_proximity_ground_values(client)
        """""
        ret2,destmm = vs.get_destination(frame)
        if ret2:
            dest = gb.convert_to_idx([coord / 10.0 for coord in destmm],2)
            dest[1]= 35-dest[1]
            dest = tuple(dest)
        
        if dest != dest_prev : 
            start = gb.convert_to_idx(center,20)
            start = tuple(start)
            path = gb.global_final(fmap,start,dest, "8N", VISUALIZE)
            state_estimation_prev = np.array([[center[0]],[center[1]], [orient], [0], [0]])
            P_estimation_prev =  np.diag([100, 100, 0.75, 10, 0.75])
            counter =0
            check_point_prev=np.array([0,0])
            start_time = time.time()
        
        else :
            dest_prev = dest
        """
        if(ground_values[0]<300 or ground_values[1]< 300):
            print('Kidnapping detected')
            await th.stop_motor(node)
            kidnapping_state= True

        if ground_values[0]>300 and ground_values[1]>300 and kidnapping_state and bool_pos:

            kidnapping_state = False
            start = gb.convert_to_idx(center,20)
            start = tuple(start)
            path = gb.global_final(fmap,start,dest, "8N", VISUALIZE)
            state_estimation_prev = np.array([[center[0]],[center[1]], [orient], [0], [0]])
            P_estimation_prev =  np.diag([100, 100, 0.75, 10, 0.75])
            counter =0
            check_point_prev=np.array([0,0])
            start_time = time.time()
     
        if not kidnapping_state:
            if x_est_cam is not None: 
                if abs(state_estimation[2][0]-x_est_cam[2] )>1:
                    x_est_cam = None
            state_estimation, P_estimation, speed, angular_speed, start_time, angle = await filtering.get_position(state_estimation_prev, P_estimation_prev, start_time,bool_pos,x_est_cam, node )
            state_estimation_prev = state_estimation
            P_estimation_prev = P_estimation

            position = np.array([state_estimation[0].item(), state_estimation[1].item()])
            theta = angle
            position = position / 20.0
            position_array = np.array(position)
            
        
            
            check_point, counter = gb.next_checkpoint2(path, position, counter,local_obstacle)
        # if theta > np.pi :
            # theta = theta-2*np.pi
        # if theta < -np.pi:
            #  theta = theta+ 2*np.pi    
                
            marre2 = np.array([position_array[0], position_array[1], theta[0]])
            check.append(marre2)
            if np.any(check_point_prev != check_point):
                print(f"Le robot est en position de {position}, converti en index {gb.convert_to_idx(position, 1)} et le prochain check point est {check_point}")
                check_point_prev = check_point
                
            if abs(position[0]-path[-1][0])<1 and abs(position[1]-path[-1][1])<1:
                await th.stop_motor(node)
                break

            angle_error=  theta-th.compute_angle(gb.convert_to_idx(position,1), path[counter])
            if angle_error > np.pi :
                angle_error = angle_error-2*np.pi
            if angle_error < -np.pi:
                angle_error = angle_error+ 2*np.pi

        
            #local nav
            #sensor values
            obscont = vs.get_obstacles(frame, robrad = 10)
            capthall = pm.hallucinate_map([position[0],position[1],(-orient)],obscont)
            sens = await th.get_proximity_values(client)
            if (sum(sens[i] > 1000 for i in range(0, 5)) > 0):#before 2500
                local_obstacle = True

            if(local_obstacle):
                print("local nav on")
                await ln.local_navigation2(client,node,[position[0],position[1],(-orient)],obscont)
            
                if(not sum(sens[i] > 1000 for i in range(0, 5)) > 0):
                    await th.motorset(node,100,100)
                    time.sleep(2)
                    local_obstacle = False
            #motor control
            else :
                if(angle_error>EPSILON_ANGLE):
                    await th.motorset(node,70,-70)
                elif (angle_error<-EPSILON_ANGLE):
                    await th.motorset(node,-70,70)
                else:
                    speed_l, speed_r = PID.PIDController(50,100, angle_error)
                    await th.motorset(node,speed_l,speed_r)
                
    await client.sleep(TS)      
   
if VISUALIZE :
    x_values = [coord[0] for coord in check]
    y_values = [coord[1] for coord in check]
    x_path =   [coord[0] for coord in path]
    y_path =  [coord[1] for coord in path]

    # Tracer le graphique x en fonction de y
    plt.plot(x_values, y_values, marker='.', linestyle='-')
    plt.plot(x_path, y_path, marker ='o', color = 'red')
    plt.title('Graphique de x en fonction de y')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.grid(True)
    plt.show()

           



    

NameError: name 'dest_prev' is not defined

In [12]:
await th.stop_motor(node)

In [7]:

## Main boucle with local nav without filtering 

local_obstacle = False
counter=0
record= []
angle =[]
check=[]
check_point_prev=np.array([0,0])
while True:
    ret,frame = cap.read()
    Tmap = vs.get_warp(cap,MAP_SHAPE,10,1)
    if ret:
        # maps capture to map
        if REFRAME:
            frame = cv2.warpPerspective(frame,Tmap,MAP_SHAPE)
        # maps BGR to HLS color space for simplicity
        HLS = cv2.cvtColor(frame, cv2.COLOR_BGR2HLS_FULL)
        obscont = vs.get_obstacles(frame, robrad = 10)
        bool_pos ,center,orient, scale = vs.get_Robot_position_orientation(HLS, 5)
        if bool_pos : 
            center = center/20.0
            center[1] = 35- center[1]
            
            x_est_cam = [center[0], center[1], orient]
            
            record.append(x_est_cam)
        #if VISUALIZE:
            #vizu = vs.visualizer(HLS)
            #omap =vs.grid_fixedmap_visualizer(fmap.transpose(),MAP_SHAPE_MM)
            #obsimg = cv2.merge([omap,omap,omap])
            #vizu = cv2.bitwise_or(vizu,obsimg)
            #vizu = vs.draw_obstacles_poly(vizu,obscont,(255,255,0),2)
            #vizu = cv2.circle(vizu,destmm,20,(50,25,100),4)
           # vizu = cv2.addWeighted(vizu,0.5,frame,0.5,0)
            #if bool_pos:
                #vizu = show_path(vizu,path,20,center)
                #vizu = vs.paint_robot(vizu,(0,0,200),center,orient,scale)
                #vizu = pm.hallucinate_map([center[0],center[1],(-orient)],obscont,vizu)
            
           # jcv2.imshow("Map",vizu)
           # if jcv2.waitKey(1) & 0xFF == ord('q'):
               # break
        if bool_pos : 
            check_point, counter = gb.next_checkpoint2(path, center, counter,local_obstacle)
            #print(counter)
            check.append(counter)
            if np.any(check_point_prev != check_point):
                print(f"Le robot est en position de {center}, converti en index {gb.convert_to_idx(center, 1)} et le prochain check point est {check_point}")
                check_point_prev = check_point
                
            if abs(center[0]-path[-1][0])<1 and abs(center[1]-path[-1][1])<1:
                await th.stop_motor(node)
                break

            angle_error=  orient-th.compute_angle(gb.convert_to_idx(center,1), path[counter])
            if angle_error > np.pi :
                angle_error = angle_error-2*np.pi
            if angle_error < -np.pi:
                angle_error = angle_error+ 2*np.pi

            #print(f"L'angle du robot est {orient} et l'angle vers le goal est {th.compute_angle(gb.convert_to_idx(center,1) , check_point)} et l'angle error est {angle_error}")
            angle.append(angle_error)
            #local nav
            #sensor values
            capthall = pm.hallucinate_map([center[0],center[1],(-orient)],obscont)
            sens = await th.get_proximity_values(client)
            if (sum(sens[i] > 500 for i in range(0, 5)) > 0):
                local_obstacle = True

            if(local_obstacle):
                print("local nav on")
                await ln.local_navigation(client,node,[center[0],center[1],(-orient)],obscont)
                
                if(not sum(sens[i] > 500 for i in range(0, 5)) > 0):
                    await th.motorset(node,100,100)
                    await client.sleep(1.5)
                    local_obstacle = False
                
            #motor control
            else :
                if(angle_error>EPSILON_ANGLE):
                    await th.motorset(node,70,-70)
                elif (angle_error<-EPSILON_ANGLE):
                    await th.motorset(node,-70,70)
                else:
                    await th.motorset(node,120,120)
            
    await client.sleep(TS)      


if VISUALIZE : 
    x_values = [coord[0] for coord in record]
    y_values = [coord[1] for coord in record]
    x_path =   [coord[0] for coord in path]
    y_path =  [coord[1] for coord in path]

    # Tracer le graphique x en fonction de y
    plt.plot(x_values, y_values, marker='.', linestyle='-')
    plt.plot(x_path, y_path, marker ='o', color = 'red')
    plt.title('Graphique de x en fonction de y')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.grid(True)
    plt.show()

        

Le robot est en position de [40.7   3.85], converti en index [40, 3] et le prochain check point est [45  8]
local nav onalid samples
Le robot est en position de [43.4  5.9], converti en index [43, 5] et le prochain check point est [39 19]
local nav on
local nav onalid samples
local nav onalid samples


CancelledError: 

In [ ]:
await th.stop_motor(node)

In [ ]:
path, state_estimation_prev2, P_estimation_prev = th.init(cap, REFRAME, MAP_SHAPE, VISUALIZE)


In [ ]:
## MAIN boucle with kalman aubin debug 

t = 0
local_obstacle = False
counter=0
record= []
angle =[]
check=[]
check_point_prev=np.array([0,0])

start_time = time.time()

norm_distance=[]
state_estimation_prev= state_estimation_prev2

while True:
    #node = await client.wait_for_node()
    ret,frame = cap.read()
    Tmap = vs.get_warp(cap,MAP_SHAPE,10,1)
    
    if ret:
        # maps capture to map
        if REFRAME:
            frame = cv2.warpPerspective(frame,Tmap,MAP_SHAPE)
        # maps BGR to HLS color space for simplicity
        HLS = cv2.cvtColor(frame, cv2.COLOR_BGR2HLS_FULL)

        bool_pos ,center,orient, scale = vs.get_Robot_position_orientation(HLS, 5)
        if bool_pos : 
            #center = center/20.0
            center[1] = 700- center[1]
            x_est_cam = np.array([center[0], center[1], orient])
            marre = np.array([x_est_cam[0]/20, x_est_cam[1]/20, orient, t])
            record.append(marre)
            #print(x_est_cam)
        else :
            x_est_cam = None
    
    else :
        x_est_cam = None
        bool_pos = False
        
    
    
   # state_estimation, P_estimation, speed, angular_speed = await filtering.get_position2(state_estimation_prev, P_estimation_prev, TS, bool_pos, x_est_cam, node)
    state_estimation, P_estimation, speed, angular_speed, start_time = await filtering.get_position(state_estimation_prev, P_estimation_prev, start_time,False,None, node, TS )
   
    state_estimation_prev = state_estimation
    P_estimation_prev = P_estimation

    position = np.array([state_estimation[0].item(), state_estimation[1].item()])
    theta = state_estimation[2]
    if theta > np.pi :
        theta = theta-2*np.pi
    if theta < -np.pi:
        theta = theta+ 2*np.pi    
    
    position = position / 20.0
    position_array = np.array(position)
    marre2 = np.array([position_array[0], position_array[1], theta[0], t])
    check.append(marre2)
    
    """""
    await th.motorset(node,120,120)
    print(f"La distance du depart est {np.sqrt((position[0]-state_estimation_prev2[0][0]/20)**2 + (position[1]-state_estimation_prev2[1][0]/20)**2)} ")
    if np.sqrt((position[0]-state_estimation_prev2[0][0]/20)**2 + (position[1]-state_estimation_prev2[1][0]/20)**2) >10:
        await th.stop_motor(node)
        break
    """
    await th.motorset(node,-70,70)
    #print(f"La distance du depart est {np.sqrt((position[0]-state_estimation_prev2[0][2]/20)**2 + (position[1]-state_estimation_prev2[1][0]/20)**2)} ")
    print (f"la diff d'angle vaut {theta -state_estimation_prev2[2][0]}")
    if t > 50:
        await th.stop_motor(node)
        break
    t= t+1       
    await client.sleep(TS)      
   
     
           



    

In [ ]:
await th.stop_motor(node)

In [ ]:
#To unlock the Robot
await node.unlock()